# Trabalho 1 de Sistemas Operacionais
## Tema 1 Entrega de Refeições

Para fazer a implementação da solução do problema proposto no tema um, utilizei o conceito de programação concorrente, que diferentemente da programação sequencial, a que estamos habituados a programar, tem por base a separação de processos dentro um mesmo programa, através de threads para otimização do processamento. Esse problema proposto, de entrega de refeições, no meu ponto de visto, é uma analogia ao problema do produtor e o consumidor, onde o produtor seria o funcionário responsável pela criação dos produtos, e o consumidor sendo o entregador, que leva o produto do ponto A até o ponto B, assim que a sacola estiver cheia. Basicamente, este problema consiste em um conjunto de processos que compartilham um mesmo buffer. Os processos chamados pela função **funcionario()** põem informação no buffer. Os processos chamados pela função **entregador()** retiram informação deste buffer.

Neste problema em questão, como sabemos, precisamos nos preocupar com acessos ilegais a certos recursos que são compartilhados entre os processos, e manter sincronismo entre os mesmos. Para controlarmos o acesso a essas variáveis e termos sincronismo nas operações, eu utilizei semáforos, que basicamente é um tipo abstrato de dados que tem como função o controle de acesso a recursos compartilhados num ambiente multitarefa. Outro controle importante é a espera ociosa que o programa irá gerar, pois quando um processo não está liberado deve entrar em estado de espera para não consumir processamento de graça, e deve ser avisado quando pode voltar a processar, com isso a ideia da analogia de acordar o entregador quando a sacola estiver cheia e voltar a dormir quando estiver vazia, e a mesma coisa com o processo do funcionário.

A biblioteca mais importante do projeto foi a threading do python, que nos proporciona a possibilidade de programação concorrente com as funções de: controle de threads, criação de threads, suspensão de threads, execução e controle de exclusão mútua por semáforos binários, para controle da seção crítica, através da função Semaphore().

Links e Referências:
Relatório: 

In [ ]:
import threading
import time
import random


semaforo = threading.Semaphore()
mutex = threading.Condition()
sacola = [] #Buffer
contador = 0
pedido = 0
itensMaximos = 10
totalProcessos = 0

#O que as threads irão copiar
def funcionario():  
  global totalProcessos #Limitador de iterações, limitando o programa a um número x processos, produções e entregas.
  while totalProcessos < 40:        
    criarProduto()

def entregador():  
  global totalProcessos  #Limita o programa a não rodar infinitamente 
  while totalProcessos < 40:       
    entregar()
  
def criarProduto(): #A função base do Funcionario     
  global totalProcessos 
  global contador
  global pedido
  menu = ["Lanche", "Pizza", "Hot-Dog", "Sushi", "Hambúrguer"]
  while True and totalProcessos < 40:
    mutex.acquire()
    semaforo.acquire() #semaforo vermelho
    aleatorio = random.randrange(5)
    if contador == itensMaximos: #Se a sacola está cheia
      semaforo.release() #semaforo verde
      mutex.wait()
      time.sleep(1)
      time.sleep(random.random())   
      return
    else:                 
      mutex.release()
      totalProcessos += 1 #técnicamente aqui também é uma seção crítica
      sacola.append(menu[aleatorio])
      pedido += 1
      contador += 1 #incremente a seção crítica
      print("Funcionario produzindo", sacola[contador -1], "- Pedidos: ", pedido)
      semaforo.release() #semaforo verde     
      time.sleep(random.random())

def entregar(): # a função base do Entragador 
  global totalProcessos 
  while True and totalProcessos < 40:
    mutex.acquire()
    semaforo.acquire() # semaforo vermelho
    global contador   
    if contador > 0:
      mutex.release()
      totalProcessos += 1
      contador -= 1 #decremente a seção crítica         
      print("Motoboy entregando", sacola[contador])
      del sacola[contador] 
      semaforo.release() #semaforo verde     
      time.sleep(random.random())      
    else:
      mutex.wait()
      time.sleep(2)
      semaforo.release() #semaforo verde     
      time.sleep(random.random())    
      return
      
if __name__ == '__main__':
  i = 0
  j = 0
  k = 0
  lucroEntregador = 0
  for i in range(30):
    for k in range(10):
      a = threading.Thread(target = funcionario, args = ())
      a.start()
    for j in range(10):
      b = threading.Thread(target = entregador, args = ())
      b.start()

  lucroEntregador = (pedido*3.5) - (7.323/5*pedido)
  print("Encerrou a noite, Motoboy entregou um total de:", pedido, "pedidos e obteve um lucro de: %.2f" %(lucroEntregador), "R$")
  

Funcionario produzindo Hot-Dog - Pedidos:  1
Funcionario produzindo Hot-Dog - Pedidos:  2
Funcionario produzindo Sushi - Pedidos:  3
Funcionario produzindo Lanche - Pedidos:  4
Funcionario produzindo Pizza - Pedidos:  5
Funcionario produzindo Sushi - Pedidos:  6
Funcionario produzindo Pizza - Pedidos:  7
Funcionario produzindo Lanche - Pedidos:  8
Funcionario produzindo Sushi - Pedidos:  9
Funcionario produzindo Sushi - Pedidos:  10
Motoboy entregando Sushi
Motoboy entregando Sushi
Motoboy entregando Lanche
Motoboy entregando Pizza
Motoboy entregando Sushi
Motoboy entregando Pizza
Motoboy entregando Lanche
Motoboy entregando Sushi
Motoboy entregando Hot-Dog
Motoboy entregando Hot-Dog
Funcionario produzindo Hambúrguer - Pedidos:  11
Funcionario produzindo Sushi - Pedidos:  12
Funcionario produzindo Hambúrguer - Pedidos:  13
Funcionario produzindo Lanche - Pedidos:  14
Funcionario produzindo Pizza - Pedidos:  15
Funcionario produzindo Hot-Dog - Pedidos:  16
Funcionario produzindo Pizza -

#Problemas encontrados e possíveis soluções
Meu maior problema foi com o sincronismo entre os processos, algumas eu deixava o semáforo vermelho em condições erradas, ou liberava ele em outras situações que acabavam tirando o sincronismo entre as threads, sendo que, acontecia de uma entregador consumir enquanto o funcionário estava produzindo, ou seja, antes da sacola ficar cheia o processo entregar era acordado, sendo tirado do seu modo de espera wait(). Algumas documentações, até mesmo no livro base da disciplina me ajudaram muito, contudo, obtive a maioria das minhas respostas no stackoverflow. Deixo aqui o link de uma explicação, da função condition() da biblioteca threading, elucidando justamente o motivo dessa função requerer uma trava. https://stackoverflow.com/questions/46076186/why-does-python-threading-condition-notify-require-a-lock

#Extra - Solução Procedural

OBS: Este tópico não fazia parte do trabalho, contudo, achei interessante mostrar a sua resolução. Sinta-se a vontade para ignorar.

In [ ]:
import time
import random

totalLanchesEntregues = 0
def funcionario():
   global totalLanchesEntregues
   if totalLanchesEntregues < 20:
     criaProduto()
   else:
     print("Entregador já bateu sua meta diária de lanches, deixe-o descansar!")
     exit(0)

def criaProduto():
  menu = ["Lanche", "Pizza", "Hot-Dog", "Sushi", "Hambúrguer"]
  escolhaMenu = []
  i = 0
  while i < 10:
    aleatorio = random.randrange(5)
    time.sleep(random.random()) 
    escolhaMenu.append(menu[aleatorio])
    print("Funcionário fazendo lanche %i, %s  " %(i + 1, escolhaMenu[i]))
    i += 1

  entregador(escolhaMenu)

def entregador(menu):
  global totalLanchesEntregues
  if totalLanchesEntregues < 20:
     fazerEntrega(menu)
  else:
     #lucroEntregador = totalLanchesEntregues*2,5 - 7,323/5*totalLanchesEntregues
     print("Sua meta diária já chegou, vá descansar! Você efetuou um total de %i entregas" %(totalLanchesEntregues))
     exit(0)
 
def fazerEntrega(menu):
  global totalLanchesEntregues
  sacola = len (menu)
  if sacola == 10:
    while sacola > 0:
      print("Entregador entregou o lanche %i, %s, #%i PEDIDO DA NOITE" %(sacola,menu[sacola-1], totalLanchesEntregues+1))
      sacola -= 1
      totalLanchesEntregues += 1 #técnicamente aqui também é uma seção crítica
      time.sleep(random.random())  
      del menu[sacola]

  funcionario()

if __name__ == '__main__':
  funcionario()
  lucroEntregador = (totalLanchesEntregues*3.5) - (7.323/5*totalLanchesEntregues)
  print("Encerrou a noite, Motoboy entregou um total de:", totalLanchesEntregues, "pedidos e obteve um lucro de: %.2f" %(lucroEntregador), "R$")

Funcionário fazendo lanche 1, Lanche  
Funcionário fazendo lanche 2, Hambúrguer  
Funcionário fazendo lanche 3, Hambúrguer  
Funcionário fazendo lanche 4, Hot-Dog  
Funcionário fazendo lanche 5, Lanche  
Funcionário fazendo lanche 6, Pizza  
Funcionário fazendo lanche 7, Hot-Dog  
Funcionário fazendo lanche 8, Pizza  
Funcionário fazendo lanche 9, Sushi  
Funcionário fazendo lanche 10, Hot-Dog  
Entregador entregou o lanche 10, Hot-Dog, #1 PEDIDO DA NOITE
Entregador entregou o lanche 9, Sushi, #2 PEDIDO DA NOITE
Entregador entregou o lanche 8, Pizza, #3 PEDIDO DA NOITE
Entregador entregou o lanche 7, Hot-Dog, #4 PEDIDO DA NOITE
Entregador entregou o lanche 6, Pizza, #5 PEDIDO DA NOITE
Entregador entregou o lanche 5, Lanche, #6 PEDIDO DA NOITE
Entregador entregou o lanche 4, Hot-Dog, #7 PEDIDO DA NOITE
Entregador entregou o lanche 3, Hambúrguer, #8 PEDIDO DA NOITE
Entregador entregou o lanche 2, Hambúrguer, #9 PEDIDO DA NOITE
Entregador entregou o lanche 1, Lanche, #10 PEDIDO DA NOITE
F